# Imports

In [ ]:
from autoboltagent.tools.low_fidelity_tool import VerboseAnalyticalTool
from autoboltagent.tools.high_fidelity_tool import VerboseFiniteElementTool

In [ ]:
from collections import deque
import numpy as np
from tqdm import tqdm

# Manual Guessing

In [ ]:
joint_config = {
    "load": 60000,
    "desired_safety_factor": 3.0,
    "bolt_yield_strength": 940,
    "plate_yield_strength": 250,
    "preload": 150000,
    "pitch": 1.5,
    "plate_thickness": 10,
    "bolt_elastic_modulus": 210,
    "plate_elastic_modulus": 210
}

In [ ]:
analytical_tool = VerboseAnalyticalTool(joint_configuration=joint_config)
fea_tool = VerboseFiniteElementTool(joint_configuration=joint_config)

In [ ]:
def cached_analytical_guess(past_guesses, num_bolts, bolt_diameter):
    result = analytical_tool.forward(num_bolts=num_bolts, bolt_diameter=bolt_diameter)
    past_guesses.appendleft((num_bolts, bolt_diameter, result))
    
    for row in past_guesses:
        print(row)

def cached_fea_guess(past_guesses, num_bolts, bolt_diameter):
    result = fea_tool.forward(num_bolts=num_bolts, bolt_diameter=bolt_diameter)
    past_guesses.appendleft((num_bolts, bolt_diameter, result))
    
    for row in past_guesses:
        print(row)

In [ ]:
past_guesses = deque()

### Usage

Simply modify the num_bolts and bolt_diameter parameters; past guesses are saved and will be printed out as well. Note that the FEA tool generates a lot of output that can't be suppressed.

In [ ]:
cached_analytical_guess(past_guesses, num_bolts=2, bolt_diameter=17)

In [ ]:
import logging
logging.basicConfig(level=logging.CRITICAL)

cached_fea_guess(past_guesses, num_bolts=2, bolt_diameter=17)

# Brute Force Search

In [ ]:
def brute_force_search(
    num_bolts_low, 
    num_bolts_high, 
    bolt_diameter_low, 
    bolt_diameter_high,
    step_size=0.1
):
    for num_bolts in range(num_bolts_low, num_bolts_high):
        for bolt_diameter in np.arange(bolt_diameter_low, bolt_diameter_high, step_size):
            result = analytical_tool.forward(num_bolts=num_bolts, bolt_diameter=float(bolt_diameter))

            if abs(result["bolt_diff"]) < 0.25 and abs(result["plate_diff"]) < 0.25:
                print(num_bolts, bolt_diameter,result)
            if result["ok"]:
                print("ANSWER ", num_bolts, bolt_diameter,result)

In [ ]:
brute_force_search(num_bolts_low=2, num_bolts_high=40, bolt_diameter_low=3.0, bolt_diameter_high=40.0)